In [ ]:

%run includes/includes.ipynb


In [ ]:

# show header
owner = owner.value
repo = repo.value
start_date = start_date.value.strftime('%Y-%m-%d')
end_date = end_date.value.strftime('%Y-%m-%d')
header_text = f"# Sprint {sprint_number.value}\n"
header_text += f"#### {start_date} to {end_date}\n"
display(Markdown(header_text))

## Sprint Activity - Commits by Author Summary

In [ ]:

%%stackql --no-display
SELECT 
    c.author, 
    COUNT(*) as commits 
FROM
    (SELECT 
    JSON_EXTRACT(commit, '$$.author.name') as author,
    DATE(DATETIME(JSON_EXTRACT(commit, '$$.author.date'), '+10 hours')) as date
    FROM github.repos.commits
    WHERE owner = '$owner' AND repo = '$repo' AND
    date > '$start_date' AND date <= '$end_date') c
GROUP BY c.author

In [ ]:

# graph commits
commit_distribution(stackql_df)

## Sprint Activity - Commits by Author Detail

In [ ]:

%%stackql --no-display
SELECT 
sha, 
JSON_EXTRACT(commit, '$$.author.name') as author,  
DATE(DATETIME(JSON_EXTRACT(commit, '$$.author.date'), '+10 hours')) as date,
TIME(DATETIME(JSON_EXTRACT(commit, '$$.author.date'), '+10 hours')) as time
FROM github.repos.commits
WHERE owner = '$owner' AND repo = '$repo' AND
date > '$start_date' AND date <= '$end_date'

In [ ]:

# save commits and display with download link 
commits_df = stackql_df
commit_shas = commits_df['sha'].tolist()
display(commits_df)
display(create_download_link(commits_df))

## Sprint Activity - Activity Detail

In [ ]:

%%stackql --no-display
SELECT 
activity_type,
date(datetime(timestamp, '+10 hours')) as date,
time(datetime(timestamp, '+10 hours')) as time,
JSON_EXTRACT(actor, '$$.login') as login
FROM
github.repos.activities
WHERE 
    owner = '$owner' AND 
    repo = '$repo'  AND
    date > '$start_date' AND date <= '$end_date'

In [ ]:

# display activity and download link
display(stackql_df)
display(create_download_link(stackql_df))

## Sprint Activity - Pull Requests

In [ ]:

%%stackql
SELECT 
    number, 
    JSON_EXTRACT(head, '$$.ref') as ref,
    date(datetime(created_at, '+10 hours')) as created_at,
    date(datetime(updated_at, '+10 hours')) as updated_at
FROM 
    github.pulls.pull_requests
WHERE 
    owner = '$owner' AND 
    repo = '$repo' AND
    created_at > '$start_date' AND created_at <= '$end_date'

## Sprint Activity - Issues

In [ ]:

%%stackql
SELECT 
title,
number,
JSON_EXTRACT(assignee, '$$.login') as assignee,
date(datetime(created_at, '+10 hours')) as created_at,
date(datetime(updated_at, '+10 hours')) as updated_at,
JSON_EXTRACT(user, '$$.login') as user,
author_association
FROM github.issues.issues
WHERE 
    owner = '$owner' AND 
    repo = '$repo' AND
    updated_at > '$start_date' AND updated_at <= '$end_date'

## Sprint Activity - Releases

In [ ]:

%%stackql
SELECT 
date(datetime(published_at, '+10 hours')) as published_at,
target_commitish,
tag_name
FROM github.repos.releases 
WHERE 
    owner = '$owner' AND 
    repo = '$repo' AND
    published_at > '$start_date' AND published_at <= '$end_date'

## Contributors - All Time

In [ ]:

%%stackql --no-display
SELECT login, contributions FROM github.repos.contributors WHERE owner = '$owner' AND repo = '$repo'

In [ ]:

# show all time contributions
show_contributions(stackql_df)

## Sprint Activity - Files

In [ ]:

# get files for each commit
queries = [
    f"""
    SELECT 
    '{sha}' as sha,
    JSON_EXTRACT(commit, '$.author.name') as author,  
    DATE(DATETIME(JSON_EXTRACT(commit, '$.author.date'), '+10 hours')) as date,
    TIME(DATETIME(JSON_EXTRACT(commit, '$.author.date'), '+10 hours')) as time,    
    JSON_EXTRACT(files.value, '$.filename') as filename,
    JSON_EXTRACT(files.value, '$.status') as status,
    JSON_EXTRACT(files.value, '$.additions') as additions,
    JSON_EXTRACT(files.value, '$.deletions') as deletions
    FROM github.repos.commits c1, json_each(files) files
    WHERE owner = '{owner}' AND repo = '{repo}' AND ref = '{sha}' 
    """
    for sha in commit_shas
]

files_df = run_stackql_queries(queries, rate_limit_per_min=15)

In [ ]:

# display files with download link
display(files_df)
display(create_download_link(files_df))